In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [5]:
df = pd.read_csv('adult.csv')

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Fit and transform the data
data_preprocessed = preprocessor.fit_transform(df)


In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x


In [7]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim, embed_dim, num_heads, ff_hidden_dim, num_layers):
        super(Generator, self).__init__()
        self.linear = nn.Linear(input_dim, embed_dim)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ff_hidden_dim) for _ in range(num_layers)
        ])
        self.output_layer = nn.Linear(embed_dim, output_dim)

    def forward(self, x):
        x = self.linear(x).unsqueeze(1)  # Add sequence dimension
        for block in self.transformer_blocks:
            x = block(x)
        x = self.output_layer(x.squeeze(1))  # Remove sequence dimension
        return x


In [8]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, ff_hidden_dim, num_layers):
        super(Discriminator, self).__init__()
        self.linear = nn.Linear(input_dim, embed_dim)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ff_hidden_dim) for _ in range(num_layers)
        ])
        self.output_layer = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.linear(x).unsqueeze(1)  # Add sequence dimension
        for block in self.transformer_blocks:
            x = block(x)
        x = self.output_layer(x.squeeze(1))  # Remove sequence dimension
        return torch.sigmoid(x)


In [10]:
from tqdm import tqdm
# Hyperparameters
input_dim = data_preprocessed.shape[1]
latent_dim = 100
embed_dim = 128
num_heads = 8
ff_hidden_dim = 256
num_layers = 3
batch_size = 64
epochs = 100

# Models
generator = Generator(latent_dim, input_dim, embed_dim, num_heads, ff_hidden_dim, num_layers)
discriminator = Discriminator(input_dim, embed_dim, num_heads, ff_hidden_dim, num_layers)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss function
criterion = nn.BCELoss()

# Training loop
for epoch in range(epochs):
    for _ in tqdm(range(data_preprocessed.shape[0] // batch_size)):
        # Train Discriminator
        idx = np.random.randint(0, data_preprocessed.shape[0], batch_size)
        real_data = torch.tensor(data_preprocessed[idx].toarray(), dtype=torch.float32)
        real_labels = torch.ones((batch_size, 1))
        fake_labels = torch.zeros((batch_size, 1))

        latent_space_samples = torch.randn((batch_size, latent_dim))
        generated_data = generator(latent_space_samples)

        real_predictions = discriminator(real_data)
        fake_predictions = discriminator(generated_data.detach())

        d_loss_real = criterion(real_predictions, real_labels)
        d_loss_fake = criterion(fake_predictions, fake_labels)
        d_loss = d_loss_real + d_loss_fake

        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        generated_data = generator(latent_space_samples)
        fake_predictions = discriminator(generated_data)

        g_loss = criterion(fake_predictions, real_labels)

        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    print(f'Epoch {epoch+1}/{epochs}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')


100%|██████████| 508/508 [00:26<00:00, 18.87it/s]


Epoch 1/100, D Loss: 0.002641930477693677, G Loss: 7.150287628173828


100%|██████████| 508/508 [00:26<00:00, 18.92it/s]


Epoch 2/100, D Loss: 0.0013189775636419654, G Loss: 7.834990501403809


100%|██████████| 508/508 [00:27<00:00, 18.46it/s]


...

Epoch 98/100, D Loss: 0.00012635687016882002, G Loss: 10.019869804382324


100%|██████████| 508/508 [00:27<00:00, 18.73it/s]


Epoch 99/100, D Loss: 0.00011392342275939882, G Loss: 10.128747940063477


100%|██████████| 508/508 [00:26<00:00, 19.16it/s]

Epoch 100/100, D Loss: 0.00010391234536655247, G Loss: 10.192365646362305


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch

# Need to redo this bit.. for inverse transformation
df = pd.read_csv('adult.csv')

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

numerical_transformer = StandardScaler()
numerical_transformer.fit(df[numerical_cols])
df[numerical_cols] = numerical_transformer.transform(df[numerical_cols])

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoded_categorical = categorical_transformer.fit_transform(df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=categorical_transformer.get_feature_names_out(categorical_cols))

# Combine preprocessed numerical and categorical data
data_preprocessed = pd.concat([df[numerical_cols].reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)
data_preprocessed_np = data_preprocessed.values

latent_dim = 100  # This should match the latent_dim used in your generator
latent_space_samples = torch.randn((data_preprocessed_np.shape[0], latent_dim))
synthetic_data = generator(latent_space_samples).detach().numpy()

# Inverse transform numerical features
synthetic_numerical_data = numerical_transformer.inverse_transform(synthetic_data[:, :len(numerical_cols)])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
# Get the synthetic categorical data
synthetic_categorical_data = synthetic_data[:, len(numerical_cols):]

# Inverse transform categorical features
inverse_transformed_categorical_data = categorical_transformer.inverse_transform(synthetic_categorical_data)

# Create DataFrames from the inverse transformed data
synthetic_numerical_df = pd.DataFrame(synthetic_numerical_data, columns=numerical_cols)
synthetic_categorical_df = pd.DataFrame(inverse_transformed_categorical_data, columns=categorical_cols)

# Concatenate numerical and categorical data
synthetic_df = pd.concat([synthetic_numerical_df, synthetic_categorical_df], axis=1)


In [18]:
synthetic_df # nonsense 🥲 (age <0)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
0,-35.414944,-287215.500000,5.968259,-5537.321289,274.682770,-18.079611,Private,12th,Never-married,Other-service,Own-child,Asian-Pac-Islander,Female,United-States,<=50K
1,67.192314,659716.625000,21.465012,13562.478516,-845.944641,24.585440,?,12th,Married-spouse-absent,Prof-specialty,Not-in-family,Amer-Indian-Eskimo,Female,United-States,>50K
2,-37.984989,-151044.859375,7.302138,-7761.911621,381.110291,-25.416937,Private,12th,Never-married,Other-service,Own-child,Amer-Indian-Eskimo,Female,United-States,<=50K
3,5.614293,-216763.046875,3.854319,-2927.934326,1186.068848,19.353485,Private,11th,Married-civ-spouse,Other-service,Husband,White,Male,United-States,>50K
4,-31.970943,-227197.015625,6.109797,-10172.156250,513.503723,-21.532618,Private,12th,Never-married,Other-service,Own-child,Amer-Indian-Eskimo,Female,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-35.840191,-239210.703125,5.383281,-8209.619141,451.426239,-23.535036,Private,12th,Never-married,Other-service,Own-child,Asian-Pac-Islander,Female,United-States,<=50K
32557,-42.788166,-171864.015625,5.860575,-8411.810547,311.722992,-25.727713,Private,12th,Never-married,Other-service,Own-child,Asian-Pac-Islander,Female,United-States,<=50K
32558,72.623451,801977.250000,21.129896,15454.623047,-842.971191,29.396048,Self-emp-not-inc,12th,Married-spouse-absent,Prof-specialty,Not-in-family,Asian-Pac-Islander,Female,United-States,>50K
32559,76.364120,736179.937500,21.841326,18654.048828,-966.969482,29.968460,?,12th,Divorced,Adm-clerical,Not-in-family,Asian-Pac-Islander,Female,United-States,>50K


In [19]:
# Saving the complete models
generator_path = 'generator_complete.pth'
discriminator_path = 'discriminator_complete.pth'

torch.save(generator, generator_path)
torch.save(discriminator, discriminator_path)

# Saving the model weights
generator_weights_path = 'generator_weights.pth'
discriminator_weights_path = 'discriminator_weights.pth'

torch.save(generator.state_dict(), generator_weights_path)
torch.save(discriminator.state_dict(), discriminator_weights_path)

In [20]:
!ls

adult.csv		    discriminator_weights.pth  generator_weights.pth
discriminator_complete.pth  generator_complete.pth     sample_data


In [ ]:
# # Loading the complete models
# generator = torch.load('generator_complete.pth')
# discriminator = torch.load('discriminator_complete.pth')

# # Loading the model weights
# # Redefine the models
# generator = Generator(latent_dim, input_dim, embed_dim, num_heads, ff_hidden_dim, num_layers)
# discriminator = Discriminator(input_dim, embed_dim, num_heads, ff_hidden_dim, num_layers)

# generator.load_state_dict(torch.load('generator_weights.pth'))
# discriminator.load_state_dict(torch.load('discriminator_weights.pth'))